In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dash
import dash_bootstrap_components as dbc
import dash_ag_grid as dag

app = dash.Dash(__name__)

ATP = pd.read_csv("atp_matches_2024.csv")
print(ATP.columns)
# print(ATP.head())
winners = ATP['winner_name'].unique()

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(dash.dcc.Dropdown(
            id='player',
            options=[{"label": name, "value": name} for name in winners],
            value=winners[0],
        ), md=4),
        dbc.Col([
            dash.dcc.Graph(
                id='chart',
                figure={},
                style={"width": "50%", "margin": "auto"}
            ),
            dash.dcc.Graph(
                id='chart2',
                figure={},
                style={"width": "50%", "margin": "auto"}
            )
        ], md=4, style={"textAlign": "center", "display": "flex"}),
        dbc.Col([
            dag.AgGrid(
                id='table',
                columnDefs=[{"field": i} for i in ATP.columns],
                rowData=[],  # Start with empty data
            )
        ], md=4)
    ])
])

@app.callback(
    dash.Output('chart', 'figure'),
    dash.Output('table', 'rowData'),
    dash.Output('table', 'defaultColDef'),
    dash.Output('chart2', 'figure'),
    dash.Input('player', 'value')
)
def update_chart(player):
    if player is None:
        return dash.no_update, dash.no_update, dash.no_update

    player_stats_win = ATP[ATP['winner_name'] == player]
    player_stats_lose = ATP[ATP['loser_name'] == player]

    win_count = len(player_stats_win)
    lose_count = len(player_stats_lose)

    # Wins by tournament
    tourney_counts_win = player_stats_win["tourney_name"].value_counts()
    # Losses by tournament
    tourney_counts_lose = player_stats_lose["tourney_name"].value_counts()

    # Align indexes for plotting
    all_tourneys = tourney_counts_win.index.union(tourney_counts_lose.index)
    win_vals = tourney_counts_win.reindex(all_tourneys, fill_value=0)
    lose_vals = tourney_counts_lose.reindex(all_tourneys, fill_value=0)

    fig = {
        "data": [
            {
                "type": "bar",
                "name": "Wins",
                "x": all_tourneys.tolist(),
                "y": win_vals.tolist(),
                "marker": {"color": "#2ECC71"},
                "opacity": 0.5
            },
            {
                "type": "bar",
                "name": "Losses",
                "x": all_tourneys.tolist(),
                "y": lose_vals.tolist(),
                "marker": {"color": "#E74C3C"},
                "opacity": 0.5
            }
        ],
        "layout": {
            "title": f"{player} - Number of Wins and Losses by Tournament (2024)",
            "xaxis": {"title": "Tournament", "tickangle": -45},
            "yaxis": {
                "title": "Number of Matches",
                "tickformat": "d"  # Show only full numbers
            },
            "margin": {"b": 120},
            "barmode": "overlay",
        }
    }
    fig2 = {
    "data": [
        {
            "type": "pie",
            "labels": ["Wins", "Losses"],
            "values": [win_count, lose_count],
            "hole": 0.7,  # optional: makes it a donut chart
            "marker": {"colors": ["#2ECC71", "#E74C3C"]}
        }
    ],
    "layout": {
        "title": f"{player} - Wins vs Losses (2024)"
    }
}
    my_cellStyle = {
        "styleConditions": [
            {
                "condition": f"params.colDef.field == '{'best_of'}' && params.value == 5",
                "style": {"backgroundColor": "lightblue", "color": "black"}
            }
        ]
    }
    return fig, player_stats_win.to_dict('records'), my_cellStyle, fig2

# player_stats_win = ATP[ATP['winner_name'] == "Holger Rune"].join(ATP[ATP['loser_name'] == "Holger Rune"], lsuffix='_winner', rsuffix='_loser', how='outer')
# plt.bar(player_stats_win["tourney_name_loser"].value_counts().keys(), player_stats_win["tourney_name_loser"].value_counts())
# plt.xticks(rotation=45, ha='right')
# plt.tight_layout()
# plt.title("Holger Rune - Number of Wins by Tournament (2024)")
# plt.show()

# print(player_stats_win[player_stats_win["surface_loser"] == "Clay"][["best_of_loser", "score_loser"]])
if __name__ == '__main__':
    app.run(debug=True)

ModuleNotFoundError: No module named 'dash_ag_grid'